In [ ]:
import pandas as pd
import os

import plotly.express as px

In [ ]:
labelalias = {
    'constant-0': 'constant-0',
    'Tanimoto-distance': 'Tanimoto',
    'Tanimoto-distance-TIMES10': 'Tanimoto_times10',
    'Tanimoto-distance-TIMES100': 'Tanimoto_times100',
    'Tanimoto-distance-EXP': 'Tanimoto_exp',
    'Tanimoto-distance-SQRT': 'Tanimoto_sqrt',
    "Tanimoto-distance-NUM_NEIGHBORS_TO_06": "Tanimoto_nn_to_06",
}



In [ ]:
eventid= '202305-2911-2320-5a95df0e-3008-4ebe-acd8-ecb3b50607c7'
input_folder = f"Runs/{eventid}"

output_folder = f'Plots/{eventid}/Cost_value_correlation'

if not os.path.exists(output_folder):
        os.makedirs(output_folder)


df_results_tot = pd.read_csv(f'{input_folder}/paroutes_n5_result_added_value_fns.csv')

In [ ]:
df_results_tot['is_solved'] = (df_results_tot['first_soln_time']!=-1 )

df_results_tot

In [ ]:
fig = px.violin(df_results_tot, x="lowest_depth_route_found", y="Tanimoto-distance", width=1000, height=600,
#              points='all',
                box=True,
                color='is_solved',
                color_discrete_sequence = [px.colors.qualitative.Plotly[1], px.colors.qualitative.Plotly[0]],
                violinmode='overlay',
             labels={
#                      "algorithm": None,
#                      "value": "Time to first solution",
#                      "species": "Species of Iris"
                 },
#              title="Time to first solution"
            )
fig.update_layout(xaxis_title="Cost of minimal-cost route to syntesise target </br></br> (assignig cost=1 to each reaction and cost=0 to each molecule in route)", 
                  yaxis_title="Tanimoto distance between taget </br></br> and closest purchasable molecule",
                  title="Correlation between nearest neighbour distance and minimal cost route"
                 )
# fig.update_xaxes(labelalias=labelalias, categoryorder='array', categoryarray=list(labelalias.keys()))
# fig.write_image(f'{output_folder}/Boxplot_time_first_solution.png') 
fig.write_image(f"{output_folder}/cost_value_correlation_Tanimoto.pdf") 
fig.show()

In [ ]:
# fig = px.box(df_results_tot, x="lowest_depth_route_found", y="Tanimoto-distance", width=1000, height=600,
#              points='all',
#              labels={
# #                      "algorithm": None,
# #                      "value": "Time to first solution",
# #                      "species": "Species of Iris"
#                  },
# #              title="Time to first solution"
#             )
# # fig.update_layout(xaxis_title=None)
# # fig.update_xaxes(labelalias=labelalias, categoryorder='array', categoryarray=list(labelalias.keys()))
# # fig.write_image(f'{output_folder}/Boxplot_time_first_solution.png') 
# fig.show()

In [ ]:

# fig = px.violin(df_results_tot, x="lowest_cost_route_(binned)", y="Tanimoto-distance", width=1000, height=600,
#              box=True,
#                 labels={
# #                      "algorithm": None,
# #                      "value": "Time to first solution",
# #                      "species": "Species of Iris"
#                  },
# #              title="Time to first solution"
#             )
# fig.update_layout(xaxis_title=None)
# fig.update_xaxes(categoryorder="category ascending")
# # fig.update_xaxes(labelalias=labelalias, categoryorder='array', categoryarray=list(labelalias.keys()))


# fig.show()